In [1]:
#IMPORTS
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import warnings



ModuleNotFoundError: No module named 'sklearn'

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# Load the dataset
data = pd.read_csv('swift_grb_catalogue.csv')

# Display the first few rows of the dataset
print(data.head())

In [ ]:
# Assuming 'T90' is the column for duration in the dataset
threshold = 2  # typically, GRBs with T90 < 2 seconds are considered short

# Create a new column 'GRB_class' where 0 = short, 1 = long
data['GRB_class'] = data['T90'].apply(lambda x: 0 if x < threshold else 1)

# Select features (you may need to adjust these depending on the dataset)
features = ['T90', 'fluence', 'peak_flux']  # example feature columns
X = data[features]
y = data['GRB_class']

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the feature columns
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Initialize the Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Predict the test set results
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

In [ ]:
# Display the classification report
print(classification_report(y_test, y_pred, target_names=['Short GRB', 'Long GRB']))